In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/concrete-data1/concrete_data.csv


In [39]:
#Importing keras modules and other necessary modules
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [40]:
#Getting the dataset from the csv file. Checking it and it's size
data = pd.read_csv("/kaggle/input/concrete-data1/concrete_data.csv")
print(data.shape)
data.head()

(1030, 9)


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [41]:
#Checking if the dataset is clean(If there are any fields that are empty or have null values)
print(data.isnull().sum())
print("="*25)

#Getting the columns of the dataset
data_col = data.columns

#Splitting the data into the independent and dependent parameters(X and y)
X = data[data_col[data_col != 'Strength']]
y = data['Strength']

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64


In [42]:
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [43]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [44]:
#Normalizing the dataset
X_norm = (X - X.mean())/X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [45]:
model = keras.Sequential([
    Dense(units=10, activation='relu', input_shape=(X.shape[1],)),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
#Splitting the data into training and testing data. (70% of data for training and 30% in testing)
def splitting_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
    return X_train, X_test, y_train, y_test

In [47]:
#Fitting the training data in the model, using the testing data on the model and
#getting the mean squared error of the result obtained from the model and the 
#actual value
def fitting(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    result = model.predict(X_test)
    error = mean_squared_error(y_test, result, squared=True)
    return error

In [48]:
#This function is to compile the process of splitting data, fitting and 
#predicting 
def fitting_and_predicting(X, y, model):
    X_train, X_test, y_train, y_test = splitting_data(X,y)
    error = fitting(X_train, X_test, y_train, y_test, model)
    return error

In [49]:
#Splitting the data into train and test datasets, fitting the data into the 
#model and obtaining the MSE between the calculated result and actual result
#Using the Normalized dataset here
MSE = []
for i in range(50):
    error = fitting_and_predicting(X_norm, y, model)
    MSE.append(error) 

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [50]:
print(MSE)

[218.81666589388243, 131.9997367484292, 99.04597530696937, 79.70770297589283, 83.66229941693462, 80.40896516245635, 70.37665255691464, 65.41759592815386, 67.24789266859541, 52.34603566842385, 37.10924961297553, 40.01931582471693, 38.87027514584472, 37.118932980649014, 39.384412176480296, 39.46103083069145, 40.07791635062449, 41.46788504599241, 35.605664723822414, 40.44836747616345, 37.27721893896968, 36.35089761670462, 36.162421985742405, 41.27648102166629, 37.17026250566405, 41.291360449490774, 36.424848871716044, 37.225137786669485, 39.87284773784657, 35.33828767283503, 39.56996737257967, 40.22557156995003, 35.49552938371921, 31.92834891409953, 37.94697626081167, 30.662931980806544, 39.90151790029937, 34.73168145366825, 36.146704813090814, 38.238882576220355, 38.98170268115713, 37.708988590003486, 36.97392981022939, 35.38480394306565, 32.28597184944136, 39.272960599993944, 37.02412737193242, 35.18330439146642, 32.87368488083698, 39.203548797696115]


In [51]:
#Calculating the mean and standard deviation of mean sqared error using numpy
MSE = np.array(MSE)
mean = np.mean(MSE)
stan_dev = np.std(MSE)
print("The mean of all the Mean Sqared Errors obtained is: {}".format(mean))
print("The Standard Deviation of all the Mean Sqared Errors obtained is: {}".format(stan_dev))


The mean of all the Mean Sqared Errors obtained is: 48.93446944445972
The Standard Deviation of all the Mean Sqared Errors obtained is: 30.998870027570845
